# Question & Answer System
by ES

In [16]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
import tarfile
import json 
import hashlib
import re
import itertools
import gensim
import string
import math
import nltk
import random
import csv
import io
import spacy
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree

ModuleNotFoundError: No module named 'spacy'

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
model = gensim.models.Word2Vec(sentences, min_count=1)
print(type(model))
# print(model.shape)

<class 'gensim.models.word2vec.Word2Vec'>


In [3]:
def toWordList(str):
    word_list = []
    words = str.split()
    for word in words:
        wordd = word.strip(',.?\'\")(').lower()
        word_list.append(wordd)
    return word_list

documents_list = {}
documents_count = {}
word_set_dic ={}
with open('documents.json','r') as f:
    documents = json.load(f)
    for dic in documents:
        word_set = set()
        para_list = []
        document_list = []
        for sentence in dic["text"]:
            word_count=Counter()
            word_list = toWordList(sentence)
            for word in word_list:
                word_count[word]+=1
                word_set.add(word)
            para_list.append(word_list)
            document_list.append(word_count)
        documents_list[dic["docid"]] = para_list
        documents_count[dic["docid"]] = document_list
        word_set_dic[dic["docid"]]=set(word_set)

# for key in documents_count:
#     print (documents_count[key])
#     print()

In [4]:
print(documents_count[410][0]['a'])

5


In [5]:
def getTF(docid,word_count):
    tfVector = []
    for word in word_set_dic[docid]:
        tfVector.append(word_count[word])
    return tfVector

def getIDF(question,docid):
    idfVector = []
    for word in word_set_dic[docid]:
        if word in question:
            count = 0
            for para in documents_list[docid]:
                if word in para:
                    count +=1
            if not count == 0:
                idfVector.append( float( len(documents_list[docid]) )/count )
            else:
                idfVector.append(0)
        else:
            idfVector.append(0)
    return idfVector

def getParagraph(question,docid):
    tfidf_list=[]
    for paraIndex in documents_count[docid]:
        tfVector = getTF(docid,paraIndex)
        idfVector = getIDF(question,docid)
        zip_list = list(zip(tfVector,idfVector))
        f = lambda x,y:x*y
        prod = sum(list(map(f,tfVector,idfVector)))
        cosine = math.sqrt(np.sum(np.square(tfVector)))*math.sqrt(np.sum(np.square(idfVector)))
        if not cosine == 0:
            tfidf_list.append(float(prod)/cosine)
        else:
            tfidf_list.append(0)
#     print (docid,np.argmax(tfidf_list),tfidf_list)
#     print(documents[docid]["text"][np.argmax(tfidf_list)])
    return documents[docid]["text"][np.argmax(tfidf_list)],np.argmax(tfidf_list)
        
    

In [6]:
with open('training.json','r') as f:
    training = json.load(f)

with open('devel.json','r') as f:
    develop = json.load(f)
    
    develop_data = []
    for question in develop:
        question_word_list = toWordList(question["question"])
        paragraph,index = getParagraph(question_word_list,question["docid"])
        develop_data.append((question["question"],paragraph,index,question["answer_paragraph"],question["text"]))
    
# with open('testing.json','r') as f:
#     testing = json.load(f)
#     answerParagraph = []
#     test_data = []
#     for question in testing:
#         question_word_list = toWordList(question["question"])
#         paragraph,index = getParagraph(question_word_list,question["docid"])
#         test_data.append((question["question"],paragraph,index))

In [7]:
def tfidfAcurate(data_list):
    data_count = 0
    correct_count = 0
    for data in data_list:
        data_count+=1
#         print(data[2],data[3])
        if data[2]==data[3]:
            correct_count +=1
    return float(correct_count)/data_count

print(tfidfAcurate(develop_data))

0.6951888924765902


In [8]:
def addEntity(sentence):
    tagged = pos_tag(word_tokenize(sentence))
    entity = nltk.chunk.ne_chunk(tagged)
    return entity

In [9]:
def getEntity(data_list):
    for data in data_list:
        question_entity = addEntity(data[0])
        paragraph_entity = addEntity(data[1])

def getAnswer(data_list):
    answer=[]
    id_num = 0
    count = [0,0,0,0,0]
    useList = [0,0]
    for data in data_list:
        question_entity = addEntity(data[0])
        paragraph_entity = addEntity(data[1])
        answer_list = []
        
        if bool(re.search('what',data[0],re.IGNORECASE)):
            count[0]+=1
            for tree in paragraph_entity:
                try:
                    if tree[1] == "NN":
                        answer_list.append(tree[0])
                except:
                    pass
        elif bool(re.search('when',data[0],re.IGNORECASE)):
            count[1]+=1
            for tree in paragraph_entity:
                try:
                    if tree.label()=="TIME":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                    if tree.label()=="DATE":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                    if tree[1] == "CD":
                        answer_list.append(tree[0])
                except:
                    pass
        elif bool(re.search('who',data[0],re.IGNORECASE)):
            count[2]+=1
            for tree in paragraph_entity:
                try:
                    if tree.label()=="PERSON":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                except:
                    pass
        elif bool(re.search('where',data[0],re.IGNORECASE)):
            count[3]+=1
            for tree in paragraph_entity:
                try:
                    if tree.label()=="LOCATION":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                    if tree.label()=="gpe":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                except:
                    pass
        else:
            count[4]+=1
            for tree in paragraph_entity:
                try:
                    if tree[1] == "NN":
                        answer_list.append(tree[0])
                except:
                    pass
        if len(answer_list)==0:
            useList[1]+=1
            index = random.randint(0,len(paragraph_entity)-1)
            try:
                answer.append (([str(id_num)+','+paragraph_entity[index][0].replace(",", "")],data[2]))
#                 print([str(id_num)+','+paragraph_entity[index][0].replace(",", "")],data[2])
            except:
                answer.append(([str(id_num)+','+paragraph_entity[index][0][0].replace(",", "")],data[2]))
#                 print([str(id_num)+','+paragraph_entity[index][0][0].replace(",", "")],data[2])
        else:
            useList[0]+=1
            index = random.randint(0,len(answer_list)-1)
#             print([str(id_num)+','+answer_list[index]],data[2])
            answer.append (([str(id_num)+','+answer_list[index].replace(",", "")],data[2]))
        id_num +=1
    print("count ",count)
    print("use list ",useList)
    return answer

    

# for line in answer:
#     print(line)
# count  [2198, 279, 422, 156, 563]
# use list  [2757, 861]

In [10]:
# answer = getAnswer(test_data)
answer = getAnswer(develop_data)

count  [1855, 163, 365, 107, 607]
use list  [2462, 635]


In [11]:
def campare(question,answer):
#     print(question,answer)
#     print(str(answer[0]).strip("'").split(',')[1][:-2])
#     print(question[4])
    
    if question[4]==str(answer[0]).strip("'").split(',')[1][:-2]:
        return True
    else:
        return False

def answerAcurate(data_list,answer):
    data_count = 0
    correct_count = 0
    results = map(campare,data_list,answer)
    for result in results:
        data_count+=1
        if result:
            correct_count +=1
    return float(correct_count)/data_count

print(answerAcurate(develop_data,answer))
    

0.004197610590894414


In [14]:
##Write into csv file
def writeFile(answer):
    # sys.stdout = io.TextIOWrapper(sys.stdout.buffer,encoding='utf8')
    out = open('answer.csv','a',newline='')
    csv_write =csv.writer(out,dialect='excel')
    csv_write.writerow(["id,answer"])
    answerID = 0
    for line in answer:
#         print(line[0])
        try:
            csv_write.writerow(line[0])
        except:
            print("line "+str(answerID)+" cannot write")
        answerID+=1
    out.close()
    
writeFile(answer)


line 730 cannot write
line 731 cannot write
line 951 cannot write
line 2149 cannot write
line 2562 cannot write
